In [1]:
!pip install -U transformers trl peft datasets evaluate rouge_score underthesea bitsandbytes thefuzz bert_score sentence-transformers
# Tải tài nguyên NLTK cho METEOR
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /home/asd/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/asd/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import os
# os.environ["WANDB_DISABLED"] = "true"
import json
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
import string
from underthesea import word_tokenize
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, EarlyStoppingCallback
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
import evaluate

2025-04-21 23:31:07.273892: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-21 23:31:07.395396: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745278267.440924 1232443 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745278267.453306 1232443 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745278267.553536 1232443 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [4]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer
import torch

def load_model_and_tokenizer(model_name, quantization="int8"):
    # Cấu hình quantization với bitsandbytes
    if quantization == "int8":
        quantization_config = BitsAndBytesConfig(
            load_in_8bit=True,  # Quantize thành INT8
            bnb_8bit_compute_dtype=torch.bfloat16,  # Dùng bfloat16 để tính toán
            bnb_8bit_use_double_quant=True,  # Double quantization để tăng hiệu quả
        )
    elif quantization == "int4":
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,  # Quantize thành INT4
            bnb_4bit_compute_dtype=torch.bfloat16,  # Dùng bfloat16 để tính toán
            bnb_4bit_use_double_quant=True,  # Double quantization
            bnb_4bit_quant_type="nf4",  # Dùng NF4 (Normalized Float 4-bit) để tối ưu
        )
    else:
        quantization_config = None  # Không quantize

    # Tải tokenizer và mô hình
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quantization_config,  # Áp dụng quantization
        torch_dtype=torch.bfloat16,
        device_map="auto",
    )

    # Cấu hình LoRA
    peft_config = LoraConfig(
        r=32,
        lora_alpha=32,
        lora_dropout=0.5,  # Tăng dropout để giảm overfitting
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "v_proj", "k_proj", "o_proj"]
    )
    model = get_peft_model(model, peft_config)
    return model, tokenizer, peft_config

# Chọn mô hình để thử (thay đổi model_name để thử từng mô hình)
model_name = "SeaLLMs/SeaLLMs-v3-1.5B-Chat" 
model, tokenizer, peft_config = load_model_and_tokenizer(model_name, quantization="int8")

tokenizer_config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/767 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/211 [00:00<?, ?B/s]

In [5]:
def formatting_func(example):
    if not all(k in example for k in ['input', 'output']):
        print('Thiếu key trong example:', example)
        return ''  # hoặc raise lỗi nếu bạn muốn dừng
    return f"<s>[INST] {example['input']} [/INST] {example['output']} </s>"

In [6]:
import os
import json
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
import string
from underthesea import word_tokenize
from thefuzz import fuzz

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))
    tokens = word_tokenize(text)
    text = " ".join(tokens)
    return text

def extract_json_from_folder(folder_path):
    dataset = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):
            file_path = os.path.join(folder_path, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    json_data = json.load(f)
                    for item in json_data:
                        if all(k in item for k in ['input', 'output']):
                            item['input'] = preprocess_text(item['input'])
                            item['output'] = preprocess_text(item['output'])
                            dataset.append(item)
                        else:
                            print(f"Thiếu trường trong {filename}: {item}")
            except json.JSONDecodeError:
                print(f"Không thể parse JSON từ {filename}")
            except Exception as e:
                print(f"Lỗi khi đọc {filename}: {e}")
    return dataset

folder_path = "/home/asd/testModel/data"
dataset = extract_json_from_folder(folder_path)

df = pd.DataFrame(dataset)
print("Số lượng giá trị duy nhất trong 'input' (exact):", df['input'].nunique())
print("Số lượng giá trị duy nhất trong 'output' (exact):", df['output'].nunique())
print("Tổng số hàng:", len(df))

# Fuzzy matching để tìm các record tương tự
similarity_threshold = 90  # Ngưỡng độ tương đồng (90%)
input_pairs = []
output_pairs = []

# Tìm các cặp input và output tương tự
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        input_sim = fuzz.ratio(df['input'].iloc[i], df['input'].iloc[j])
        if input_sim >= similarity_threshold:
            input_pairs.append((i, j, input_sim))
        output_sim = fuzz.ratio(df['output'].iloc[i], df['output'].iloc[j])
        if output_sim >= similarity_threshold:
            output_pairs.append((i, j, output_sim))

print(f"Số cặp input tương tự (>{similarity_threshold}%):", len(input_pairs))
print(f"Số cặp output tương tự (>{similarity_threshold}%):", len(output_pairs))

# Loại bỏ các record có input hoặc output tương tự (giữ record đầu tiên)
indices_to_keep = set(range(len(df)))
for i, j, _ in input_pairs:
    if j in indices_to_keep:
        indices_to_keep.remove(j)
for i, j, _ in output_pairs:
    if j in indices_to_keep:
        indices_to_keep.remove(j)

df = df.iloc[list(indices_to_keep)].reset_index(drop=True)
print("Số hàng sau khi xóa record tương tự:", len(df))

# Kiểm tra lại độ unique
print("Số lượng giá trị duy nhất trong 'input' (sau xử lý):", df['input'].nunique())
print("Số lượng giá trị duy nhất trong 'output' (sau xử lý):", df['output'].nunique())

# Chia train/validation
full_dataset = Dataset.from_pandas(df[['input', 'output']])
train_df, eval_df = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_df[['input', 'output']])
eval_dataset = Dataset.from_pandas(eval_df[['input', 'output']])
print(train_df[['input', 'output']].head())

Số lượng giá trị duy nhất trong 'input' (exact): 1599
Số lượng giá trị duy nhất trong 'output' (exact): 1659
Tổng số hàng: 1700
Số cặp input tương tự (>90%): 320
Số cặp output tương tự (>90%): 41
Số hàng sau khi xóa record tương tự: 1471
Số lượng giá trị duy nhất trong 'input' (sau xử lý): 1471
Số lượng giá trị duy nhất trong 'output' (sau xử lý): 1471
                                                  input  \
998   trẻ không biết tôn trọng lượt nói trong thảo l...   
254                             trẻ nói ít phải làm sao   
1073  trẻ không biết chuẩn bị nội dung khi báo cáo nhóm   
643   trẻ không biết nói lời chia tay khi bạn chuyển...   
1450  tự kỷ có học được kỹ năng kết thúc tương tác x...   

                                                 output  
998   nói chen hoặc ngắt lời bạn là thiếu kỹ năng xã...  
254   cần đánh giá xem trẻ có hiểu lời tương tác bằn...  
1073  nội dung rời rạc là thiếu chuẩn bị – nên luyện...  
643   không thể hiện chia tay là thiếu kỹ năng chia ...  


In [7]:
import evaluate
import numpy as np
from sentence_transformers import SentenceTransformer, util
def evaluate_metrics(predictions, references):
    # Tải các độ đo
    rouge = evaluate.load("rouge")
    meteor = evaluate.load("meteor")

    # Tính các độ đo
    rouge_results = rouge.compute(predictions=predictions, references=references)
    meteor_results = meteor.compute(predictions=predictions, references=references)

    # Tải mô hình nhúng câu để tính Cosine Similarity
    embedder = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

    # Tính embeddings cho dự đoán và tham chiếu
    pred_embeddings = embedder.encode(predictions, convert_to_tensor=True)
    ref_embeddings = embedder.encode(references, convert_to_tensor=True)

    # Tính Cosine Similarity giữa từng cặp dự đoán-tham chiếu
    cosine_scores = util.cos_sim(pred_embeddings, ref_embeddings)
    # Lấy trung bình Cosine Similarity (chỉ lấy đường chéo chính, vì mỗi dự đoán chỉ so với tham chiếu tương ứng)
    avg_cosine_similarity = np.mean([cosine_scores[i][i].item() for i in range(len(predictions))])
    
    # Gộp kết quả
    metrics = {
        "rouge1": rouge_results["rouge1"],
        "rouge2": rouge_results["rouge2"],
        "rougeL": rouge_results["rougeL"],
        "meteor": meteor_results["meteor"],
        "cosine_similarity": avg_cosine_similarity,
    }
    return metrics

def generate_predictions(model, tokenizer, inputs, max_length=200):
    """Tạo dự đoán từ mô hình cho các đầu vào."""
    model.eval()
    predictions = []
    # Đảm bảo pad_token được thiết lập
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    for input_text in inputs:
        # Tiền xử lý input để đồng bộ với huấn luyện
        input_text = preprocess_text(input_text)
        prompt = f"<s>[INST] {input_text} [/INST]"
        inputs_encoded = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs_encoded,
                max_length=max_length,
                num_return_sequences=1,
                pad_token_id=tokenizer.pad_token_id
            )
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        pred = generated_text.split("[/INST]")[-1].strip()
        # Tiền xử lý dự đoán để đồng bộ với tham chiếu
        pred = preprocess_text(pred)
        predictions.append(pred)
    return predictions

In [8]:
import json
import time
from transformers import TrainingArguments
from trl import SFTTrainer
from transformers import EarlyStoppingCallback

# Loại bỏ cột không cần thiết để tránh cảnh báo
train_dataset = train_dataset.remove_columns(['__index_level_0__'] if '__index_level_0__' in train_dataset.column_names else [])
eval_dataset = eval_dataset.remove_columns(['__index_level_0__'] if '__index_level_0__' in eval_dataset.column_names else [])

# Đo thời gian chạy đơn
print("\n=== Bắt đầu chạy đơn ===")
start_time = time.time()
# Cấu hình huấn luyện cho Single Run
training_arguments_single = TrainingArguments(
    output_dir="./results_single_seaLLM",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=10,
    learning_rate=5e-5,
    weight_decay=0.1,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.1,
    group_by_length=True,
    lr_scheduler_type="cosine",
    eval_strategy="steps",
    eval_steps=10,
    logging_strategy="steps",
    log_level="info",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

# Huấn luyện Single Run
trainer_single = SFTTrainer(
    model=model,
    args=training_arguments_single,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    formatting_func=formatting_func,
    callbacks=[
        EarlyStoppingCallback(
            early_stopping_patience=3,
            early_stopping_threshold=0.01,
        )
    ],
)
print(f"Train dataset size: {len(train_dataset)}, Eval dataset size: {len(eval_dataset)}")
trainer_single.train()

# Lưu mô hình Single Run
model.save_pretrained("./finetuned_seaLLM_single")
tokenizer.save_pretrained("./finetuned_seaLLM_single")

# Đánh giá Single Run
test_inputs = eval_df['input'].tolist()
test_references = eval_df['output'].tolist()
predictions_single = generate_predictions(model, tokenizer, test_inputs)
metrics_single = evaluate_metrics(predictions_single, test_references)
print("Single Run Metrics:", metrics_single)

# Lưu metrics vào file
with open("single_run_metrics.json", "w") as f:
    json.dump(metrics_single, f, indent=4)

end_time = time.time()
duration = end_time - start_time
print(f"Thời gian chạy đơn: {duration:.2f} giây")
# Kiểm tra mẫu dự đoán
for i in range(5):
    print(f"Input: {test_inputs[i]}")
    print(f"Prediction: {predictions_single[i]}")
    print(f"Reference: {test_references[i]}\n")


=== Bắt đầu chạy đơn ===


Applying formatting function to train dataset:   0%|          | 0/1176 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/1176 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/1176 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1176 [00:00<?, ? examples/s]

...

Train dataset size: 1176, Eval dataset size: 295


/home/asd/testModel/.conda/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
10,3.539100,3.846084
20,3.741000,3.505600
30,3.216000,3.086502
40,2.850600,2.660569
50,2.527700,2.350333
60,2.328400,2.232741
70,2.199000,2.148887
80,2.054500,2.104634
90,2.098100,2.036708
100,1.973600,2.004707


The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: output, input, text. If output, input, text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 295
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: output, input, text. If output, input, text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 295
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: output, input, text. If output, input, text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 2

Single Run Metrics: {'rouge1': np.float64(0.5591751945683279), 'rouge2': np.float64(0.1893472342082259), 'rougeL': np.float64(0.3304493997887862), 'meteor': np.float64(0.17135916127952353), 'cosine_similarity': np.float64(0.6145371492636406)}
Thời gian chạy đơn: 777.00 giây
Input: trẻ không biết tôn trọng ranh giới của người khác
Prediction: không biết giữ khoảng cách riêng tư cho bản thân nên có thể gây ra việc quấy rối hoặc xúc phạm không được phép s
Reference: cố ép bạn làm theo ý mình là chưa có kỹ năng đồng thuận – nên luyện hỏi bạn có muốn chơi thế này không trước khi bắt đầu

Input: trẻ không dùng tay để chỉ vật
Prediction: việc thiếu kỹ năng sử dụng ngón tay để chỉ đồ vật là biểu hiện chậm phát triển ngôn ngữ – nên luyện bằng cách cầm mô hình vật từ từ s
Reference: việc trẻ không dùng tay để chỉ khi muốn thể hiện nhu cầu là dấu hiệu quan trọng để nhận biết trẻ có khó khăn trong giao tiếp phi ngôn ngữ

Input: tự kỷ và rối loạn ngôn ngữ khác gì nhau
Prediction: tự kỷ là một rối l

In [9]:
# Cell 8: 5-Fold Cross-Validation
from sklearn.model_selection import KFold
import numpy as np
import gc

# Cấu hình KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_metrics = []
fold_models = []
fold_times = []
total_start_time = time.time()

# Lặp qua từng fold
for fold, (train_idx, eval_idx) in enumerate(kf.split(df)):
    print(f"\nTraining Fold {fold + 1}...")
    fold_start_time = time.time()
    
    # Tạo tập train và eval cho fold hiện tại
    train_fold = df.iloc[train_idx][['input', 'output']]
    eval_fold = df.iloc[eval_idx][['input', 'output']]
    train_fold_dataset = Dataset.from_pandas(train_fold)
    eval_fold_dataset = Dataset.from_pandas(eval_fold)

    # Loại bỏ cột không cần thiết
    train_fold_dataset = train_fold_dataset.remove_columns(['__index_level_0__'] if '__index_level_0__' in train_fold_dataset.column_names else [])
    eval_fold_dataset = eval_fold_dataset.remove_columns(['__index_level_0__'] if '__index_level_0__' in eval_fold_dataset.column_names else [])

    # Tải lại mô hình gốc với INT4 quantization
    model, tokenizer, peft_config = load_model_and_tokenizer(model_name,quantization="int4")
    print(f"Fold {fold + 1} - Train size: {len(train_fold_dataset)}, Eval size: {len(eval_fold_dataset)}")
    # Cấu hình huấn luyện cho fold
    training_arguments_fold = TrainingArguments(
        output_dir=f"./results_fold_{fold + 1}",
        num_train_epochs=3,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        optim="paged_adamw_32bit",
        save_steps=100,
        logging_steps=10,
        learning_rate=5e-5,
        weight_decay=0.1,
        fp16=False,
        bf16=True,
        max_grad_norm=0.3,
        warmup_ratio=0.1,
        group_by_length=True,
        lr_scheduler_type="cosine",
        eval_strategy="steps",
        eval_steps=10,
        logging_strategy="steps",
        log_level="info",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
    )

    # Huấn luyện fold
    trainer_fold = SFTTrainer(
        model=model,
        args=training_arguments_fold,
        train_dataset=train_fold_dataset,
        eval_dataset=eval_fold_dataset,
        peft_config=peft_config,
        formatting_func=formatting_func,
        callbacks=[
            EarlyStoppingCallback(
                early_stopping_patience=3,
                early_stopping_threshold=0.01,
            )
        ],
    )
    trainer_fold.train()

    # Lưu mô hình fold
    fold_path = f"./finetuned_seaLLM_fold_{fold + 1}"
    model.save_pretrained(fold_path)
    tokenizer.save_pretrained(fold_path)
    fold_models.append(fold_path)

    # Đánh giá fold
    test_inputs_fold = eval_fold['input'].tolist()
    test_references_fold = eval_fold['output'].tolist()
    predictions_fold = generate_predictions(model, tokenizer, test_inputs_fold)
    metrics_fold = evaluate_metrics(predictions_fold, test_references_fold)
    print(f"Fold {fold + 1} Metrics:", metrics_fold)
    fold_metrics.append(metrics_fold)

    # Lưu metrics của fold
    with open(f"fold_{fold + 1}_metrics.json", "w") as f:
        json.dump(metrics_fold, f, indent=4)

    fold_end_time = time.time()
    fold_duration = fold_end_time - fold_start_time
    fold_times.append(fold_duration)
    print(f"Thời gian chạy Fold {fold + 1}: {fold_duration:.2f} giây")
    
    # Dọn dẹp bộ nhớ
    del model, trainer_fold
    torch.cuda.empty_cache()
    gc.collect()

# Tính tổng thời gian và thời gian trung bình
total_end_time = time.time()
total_duration = total_end_time - total_start_time
print(f"\n=== Kết thúc huấn luyện K-Fold ===")
print(f"Tổng thời gian chạy: {total_duration:.2f} giây")
print(f"Thời gian trung bình mỗi fold: {np.mean(fold_times):.2f} giây")

# Tính trung bình metrics qua các fold
avg_metrics = {
    "rouge1": np.mean([m["rouge1"] for m in fold_metrics]),
    "rouge2": np.mean([m["rouge2"] for m in fold_metrics]),
    "rougeL": np.mean([m["rougeL"] for m in fold_metrics]),
    "meteor": np.mean([m["meteor"] for m in fold_metrics]),
    "cosine_similarity": np.mean([m["cosine_similarity"] for m in fold_metrics]),
}
print("\nAverage Cross-Validation Metrics:", avg_metrics)

# Lưu metrics trung bình
with open("cross_validation_seaLLM_metrics.json", "w") as f:
    json.dump(avg_metrics, f, indent=4)


Training Fold 1...


loading file vocab.json from cache at /home/asd/.cache/huggingface/hub/models--SeaLLMs--SeaLLMs-v3-1.5B-Chat/snapshots/560ccd5b29391001dcd28ae9e5b3ead80ac76114/vocab.json
loading file merges.txt from cache at /home/asd/.cache/huggingface/hub/models--SeaLLMs--SeaLLMs-v3-1.5B-Chat/snapshots/560ccd5b29391001dcd28ae9e5b3ead80ac76114/merges.txt
loading file tokenizer.json from cache at /home/asd/.cache/huggingface/hub/models--SeaLLMs--SeaLLMs-v3-1.5B-Chat/snapshots/560ccd5b29391001dcd28ae9e5b3ead80ac76114/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /home/asd/.cache/huggingface/hub/models--SeaLLMs--SeaLLMs-v3-1.5B-Chat/snapshots/560ccd5b29391001dcd28ae9e5b3ead80ac76114/tokenizer_config.json
loading file chat_template.jinja from cache at None
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
loading co

Fold 1 - Train size: 1176, Eval size: 295


loading file vocab.json from cache at /home/asd/.cache/huggingface/hub/models--SeaLLMs--SeaLLMs-v3-1.5B-Chat/snapshots/560ccd5b29391001dcd28ae9e5b3ead80ac76114/vocab.json
loading file merges.txt from cache at /home/asd/.cache/huggingface/hub/models--SeaLLMs--SeaLLMs-v3-1.5B-Chat/snapshots/560ccd5b29391001dcd28ae9e5b3ead80ac76114/merges.txt
loading file tokenizer.json from cache at /home/asd/.cache/huggingface/hub/models--SeaLLMs--SeaLLMs-v3-1.5B-Chat/snapshots/560ccd5b29391001dcd28ae9e5b3ead80ac76114/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /home/asd/.cache/huggingface/hub/models--SeaLLMs--SeaLLMs-v3-1.5B-Chat/snapshots/560ccd5b29391001dcd28ae9e5b3ead80ac76114/tokenizer_config.json
loading file chat_template.jinja from cache at None
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Applying formatting function to train dataset:   0%|          | 0/1176 [00:00<?, ? examples/s]

...

Truncating eval dataset:   0%|          | 0/294 [00:00<?, ? examples/s]

Using auto half precision backend
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: output, input, text. If output, input, text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1,177
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 8
  Total optimization steps = 219
  Number of trainable parameters = 8,716,288


Step,Training Loss,Validation Loss
10,3.655600,3.885910
20,3.838300,3.605725
30,3.349800,3.186525
40,2.987600,2.845254
50,2.721900,2.645740
60,2.478000,2.539057
70,2.463600,2.447129
80,2.622600,2.277462
90,2.161100,2.167216
100,2.073800,2.076878


The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: output, input, text. If output, input, text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 294
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: output, input, text. If output, input, text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 294
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: output, input, text. If output, input, text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 2

Fold 5 Metrics: {'rouge1': np.float64(0.5677152500577425), 'rouge2': np.float64(0.20283771281892216), 'rougeL': np.float64(0.3349405869660811), 'meteor': np.float64(0.19715111287253426), 'cosine_similarity': np.float64(0.6436814488336241)}
Thời gian chạy Fold 5: 458.82 giây

=== Kết thúc huấn luyện K-Fold ===
Tổng thời gian chạy: 2207.61 giây
Thời gian trung bình mỗi fold: 441.13 giây

Average Cross-Validation Metrics: {'rouge1': np.float64(0.5644927812158846), 'rouge2': np.float64(0.19762640983969276), 'rougeL': np.float64(0.3338656403233208), 'meteor': np.float64(0.18973665620386798), 'cosine_similarity': np.float64(0.6318929949952303)}


In [ ]:
import torch
import gc
import copy
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Giải phóng bộ nhớ trước
torch.cuda.empty_cache()
gc.collect()

# Hàm tải mô hình gốc
def load_model_and_tokenizer(quantization="int8"):
    model_name = ""
    quantization_config = BitsAndBytesConfig(
        load_in_8bit=True,
        bnb_8bit_compute_dtype=torch.bfloat16,
        bnb_8bit_use_double_quant=True,
    ) if quantization == "int8" else None

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quantization_config,
        torch_dtype=torch.bfloat16,
        device_map="auto"
    )
    return model, tokenizer, None

# Tải mô hình gốc với INT8 quantization
base_model, tokenizer, _ = load_model_and_tokenizer(quantization="int8")

# Đường dẫn đến fold 4
fold_4_path = "./finetuned_vistral_fold_4"

# Tải mô hình fine-tune fold 4
fold_4_model = PeftModel.from_pretrained(base_model, fold_4_path, is_trainable=False)
fold_4_state_dict = copy.deepcopy(fold_4_model.state_dict())

# Kết hợp trọng số (50% mô hình gốc + 50% fold 4)
base_state_dict = copy.deepcopy(base_model.state_dict())
combined_state_dict = {}
for key in base_state_dict:
    if key in fold_4_state_dict:
        combined_state_dict[key] = 0.5 * base_state_dict[key] + 0.5 * fold_4_state_dict[key]
    else:
        combined_state_dict[key] = base_state_dict[key]

# Tải trọng số kết hợp vào mô hình gốc
base_model.load_state_dict(combined_state_dict, strict=False)

# Lưu mô hình kết hợp
combined_model_path = "./finetuned_vistral_combined_fold_4"
base_model.save_pretrained(combined_model_path)
tokenizer.save_pretrained(combined_model_path)

# Dọn dẹp bộ nhớ
del base_model
del fold_4_model
del base_state_dict
del fold_4_state_dict
del combined_state_dict
torch.cuda.empty_cache()
gc.collect()

TypeError: load_model_and_tokenizer() missing 1 required positional argument: 'model_name'